# download corpus

In [2]:
# 터미널에서 사용할 것은 apt로 설치
#!apt install -y curl

In [3]:
#!ls
#!mkdir data

# small wiki corpus
#!curl -c ./data -s -L "https://drive.google.com/uc?export=download&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" > /dev/null
#!curl -Lb ./data "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" -o data/wiki_20190620_small.txt

# all kor wiki corpus
#!curl -c ./data -s -L "https://drive.google.com/uc?export=download&id=1_F5fziHjUM-jKr5Pwcx1we6g_J2o70kZ" > /dev/null
#!curl -Lb ./data "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1_F5fziHjUM-jKr5Pwcx1we6g_J2o70kZ" -o data/wiki_20190620.txt


# generate tokenizer(generate vocab)

In [1]:
#!mkdir custom_tokenizers

# tokenizer models: bpe, unigram, wordlevel, wordpiece
from tokenizers import BertWordPieceTokenizer

In [2]:
# init
wp_tokenizer = BertWordPieceTokenizer(
    clean_text=True,           # 공백 제거
    handle_chinese_chars=True, # 한자는 하나가 한 토큰
    strip_accents=False,      # if True, [StripAccents] -> [Strip, Accents]
    lowercase=False,           # if True, LowerCase -> lowercase
)

# train
wp_tokenizer.train(
    files="/opt/ml/other/BERT_pretrain/data/wiki_20190620_small.txt",
    vocab_size=20000,
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    wordpieces_prefix="##",
)

# save
wp_tokenizer.save_model("custom_tokenizers", "wordpiece_tokenizer")

['custom_tokenizers/wordpiece_tokenizer-vocab.txt']

In [20]:
# check
text = "뷁은 [MASK] 중기의 무신이다."
tokenized_text = wp_tokenizer.encode(text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

['[UNK]', '[MASK]', '중', '##기의', '무신', '##이다', '.']
[1, 4, 755, 2604, 13161, 1895, 16]


# create tokenizer (load vocab)

In [25]:
import torch
from transformers import BertConfig, BertForPreTraining, BertTokenizerFast
device = torch.device("cuda")

In [35]:
# custom tokenizer
tokenizer = BertTokenizerFast(
    vocab_file="/opt/ml/other/BERT_pretrain/custom_tokenizers/wordpiece_tokenizer-vocab.txt",
    max_len=128,
    strip_accents=False,
    lowercase=False,
)

# check
print(tokenizer.tokenize("뷁은 [MASK] 중기의 무신이다."))

['[UNK]', '[', 'ma', '##s', '##k', ']', '중', '##기의', '무신', '##이다', '.']


In [32]:
# add [MASK] token
tokenizer.add_special_tokens({'mask_token': '[MASK]'})

# check
print(tokenizer.tokenize("뷁은 [MASK] 중기의 무신이다."))

['[UNK]', '[MASK]', '중', '##기의', '무신', '##이다', '.']


# init model

In [10]:
from transformers import BertConfig, BertForPreTraining

# customize default BERT config
# https://huggingface.co/transformers/model_doc/bert.html#bertconfig
config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    max_position_embeddings=tokenizer.model_max_length, # 최대 token 개수. (BERT default=512)
    #type_vocab_size=2, # token type id 개수 (BERT는 segmentA, segmentB로 2종류)
    #position_embedding_type="absolute"
)

# init model
# for pretraining oneself, use ModelForPreTraining(config)
model = BertForPreTraining(config=config)
model.num_parameters()

101720098

# create dataset

In [12]:
import os
import random
import warnings
from time import time
from filelock import FileLock
from typing import Dict, List, Optional

import json
import pickle

import torch
from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers import DataCollatorForLanguageModeling
from transformers.utils import logging

logger = logging.get_logger(__name__)

In [41]:
class TextDatasetForNextSentencePrediction(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(self,
        tokenizer: PreTrainedTokenizer,
        file_path,
        block_size,
        short_seq_prob=0.1,
        nsp_prob=0.5,
        overwrite_cache=False,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        self.tokenizer = tokenizer
        self.block_size = block_size - tokenizer.num_special_tokens_to_add(pair=True)
        self.short_seq_prob = short_seq_prob
        self.nsp_prob = nsp_prob

        # 학습 데이터 caching
        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            directory,
            "cached_nsp_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        lock_path = cached_features_file + ".lock"

        # Input file format:
        # (1) One sentence per line.
        # - Must a sentence, not paragraphs or so.
        # - Because the task is "next sentence prediction".
        # (2) Blank lines between documents.
        # - Document boundaries are needed.
        # - So that the "next sentence prediction" task doesn't span between documents.
        #
        # Example:
        # I am very happy.
        # Here is the second sentence.
        #
        # A new document.

        with FileLock(lock_path):
            # cached data가 존재하면, dataset을 생성할 필요없이 그대로 사용
            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file}"
                    f"[took {time()-start: .3f} sec]"
                )
            # 그렇지 않으면, dataset을 만들어서 사용
            else:
                # corpus를 load하여 document 별로 grouping(형식화)
                logger.info(f"Creating features from dataset file at {directory}")
                self.documents = [[]]
                with open(file_path, encoding="utf-8") as f:
                    while True:
                        line = f.readline()
                        # 종료 조건: 더 이상 line이 없으면 break
                        if not line:
                            break
                        
                        # 메인 내용
                        # blank line이 나오면 line들을 document로 취합
                        # black line은 strip했을 때, not line
                        line = line.strip()
                        if not line and len(self.documents[-1]) != 0:
                            self.documents.append([])
                        tokens = tokenizer.tokenize(line)
                        tokens = tokenizer.convert_tokens_to_ids(tokens)
                        if tokens:
                            self.documents[-1].append(tokens)
                
                # documents로 examples 생성(학습에 맞는 데이터로 변형)
                logger.info(f"Creating examples from {len(self.documents)} documents.")
                self.examples = []
                for doc_index, document in enumerate(self.documents):
                    self.create_examples_from_document(document, doc_index)

                start = time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    f"Saving features into cached file {cached_features_file}"
                    f"[took {time()-start: .3f} sec]"
                )

    def create_examples_from_document(self, document: List[List[int]], doc_index: int):
        """Creates examples for a single document."""
        # 최대 토큰 수 = embedding size - 고정적으로 사용하는 special token 개수
        # 여기서는 [CLS], [SEP] token이 부착되기 때문에, 2 만큼 빼줍니다.
        # 예를 들어 embedding size가 128이면, 학습 데이터로부터 최대 126개의 token만 가져옵니다.
        max_num_tokens = self.block_size - self.tokenizer.num_special_tokens_to_add(pair=True)

        # 문장 길이의 일반화를 위해서 short_seq_prob 비율의 데이터는 짧은 길이의 문장으로 만듭니다.
        # 예를 들어 max_num_tokens가 126이면, 2이상 126이하의 범위에서 랜덤하게 길이를 갖습니다.
        target_seq_length = max_num_tokens
        if random.random() < self.short_seq_prob:
            target_seq_length = random.randint(2, max_num_tokens)

        # document의 segment들을 합쳐서 chunk로 만듭니다.
        # chunk는 seg1[SEP]seg2처럼 무조건 두 개를 합치는 것이 아니라,
        # 126 token을 꽉 채우기 위해 seg1+seg2[SEP]seg3+seg4처럼 여러 개를 합칠 수 있습니다.
        # 결과적으로는 tokens1[SEP]tokens2의 형태가 됩니다.
        current_chunk = []
        current_length = 0
        i = 0
        while i < len(document):
            segment = document[i]
            current_chunk.append(segment)
            current_length += len(segment)

            # 마지막 segment이거나 chunk 길이가 목표 길이 이상인 경우
            if i == len(document) - 1 or current_length >= target_seq_length:
                # chunk를 두 부분(tokens_a, tokens_b)으로 나눕니다.
                if current_chunk:
                    # chunk 길이가 2 이상이면, 앞 부분을 랜덤하게 자릅니다.
                    # 앞 부분의 길이는 2 이상 현재 chunk 길이 이하입니다.
                    # (따라서 인덱스는 1 이상 현재 chunk 길이 - 1 이하)
                    a_end = 1 # tokens_a의 마지막 인덱스
                    if len(current_chunk) >= 2:
                        a_end = random.randint(1, len(current_chunk) - 1)
                    tokens_a = []
                    for j in range(a_end):
                        tokens_a.extend(current_chunk[j])
                    
                    # [SEP] 뒷 부분인 tokens_b를 결정합니다.
                    # tokens_b 길이 = 전체 길이 - tokens_a 길이
                    # tokens_b의 segments는 nsp_prob의 확률로 결정됩니다.
                    # - nsp_prob의 확률로 랜덤하게 다른 문장을 선택합니다.
                    # - (1-nsp_prob)의 확률로 다음 문장을 선택합니다.
                    tokens_b = []
                    # 랜덤하게 다른 문장을 선택하는 부분
                    if len(current_chunk) == 1 or random.random() < self.nsp_prob:
                        is_random_next = True
                        target_b_length = target_seq_length - len(tokens_a)

                        # 랜덤하게 선택된 doc이 원래 doc과 같지 않도록 합니다.
                        # 보통은 한 번에 끝나겠지만, 만약을 위해 최대 10번 시도하도록 합니다.
                        for _ in range(10):
                            random_document_index = random.randint(0, len(self.documents) - 1)
                            if random_document_index != doc_index:
                                break
                        # 선택된 랜덤 index로 document를 가져옵니다.
                        random_document = self.documents[random_document_index]
                        random_start = random.randint(0, len(random_document) - 1)
                        for j in range(random_start, len(random_document)):
                            tokens_b.extend(random_document[j])
                            if len(tokens_b) >= target_b_length:
                                break
                        
                        # 잘려서 사용되지 않은 부분은 버려지지 않도록 다시 넣어 놓습니다.
                        num_unused_segments = len(current_chunk) - a_end
                        i -= num_unused_segments
                    # 실제 다음 문장을 선택하는 부분
                    else:
                        is_random_next = False
                        for j in range(a_end, len(current_chunk)):
                            tokens_b.extend(current_chunk[j])

                    def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens):
                        """Truncates a pair of sequences to a maximum sequence length."""
                        while True:
                            total_length = len(tokens_a) + len(tokens_b)
                            # 종료 조건: 최대 토큰수 이하면 trunc를 종료
                            if total_length <= max_num_tokens:
                                break

                            # tokens_a, b 중 더 긴 것을 trunc합니다.
                            trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
                            assert len(trunc_tokens) >= 1

                            # 랜덤하게 앞에서 혹은 뒤에서 trunc합니다.
                            # 비율은 0.5로 하여 bias를 방지합니다.
                            if random.random() < 0.5:
                                del trunc_tokens[0] # 앞에서 token 하나 제거
                            else:
                                trunc_tokens.pop() # 뒤에서 token 하나 제거

                    truncate_seq_pair(tokens_a, tokens_b, max_num_tokens)

                    assert len(tokens_a) >= 1
                    assert len(tokens_b) >= 1

                    # add special tokens
                    input_ids = self.tokenizer.build_inputs_with_special_tokens(tokens_a, tokens_b)
                    # add token type ids, 0 for sentence a, 1 for sentence b
                    token_type_ids = self.tokenizer.create_token_type_ids_from_sequences(tokens_a, tokens_b)
                    
                    # 드디어 아래 항목에 대한 데이터셋이 만들어졌습니다! :-)
                    # 즉, segmentA[SEP]segmentB, [0, 0, .., 0, 1, 1, ..., 1], NSP 데이터가 만들어진 것입니다 :-)
                    # 그럼 다음은.. 이 데이터에 [MASK] 를 씌워야겠죠?
                    example = {
                        "input_ids": torch.tensor(input_ids, dtype=torch.long),
                        "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
                        "next_sentence_label": torch.tensor(1 if is_random_next else 0, dtype=torch.long),
                    }

                    self.examples.append(example)

                current_chunk = []
                current_length = 0

            i += 1

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

In [42]:
# dataset using small wiki corpus
dataset = TextDatasetForNextSentencePrediction(
    tokenizer=tokenizer,
    file_path="/opt/ml/other/BERT_pretrain/data/wiki_20190620_small.txt",
    block_size=128,
    overwrite_cache=False,
    short_seq_prob=0.1,
    nsp_prob=0.5,
)

# data collator
# [MASK] 씌우는 부분은 직접 구현할 필요 없다.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [44]:
# check dataset
for example in dataset.examples[0:1]:
    print(example)

{'input_ids': tensor([    2,  2492,  2429,  2780,  1969,  5380,  3118,  1941,  2408,    16,
         5498, 10307, 16245,   555,  1242,   822,  1389,  1082,   930, 16489,
        12287,  1124,  3667,    16,  6533,  8935,  1016,  2678,  1907,    16,
          176,   984,  4021,  1014,  8599,   729,  1167,    93,  7745,    93,
        10414,  1006, 18366,  3486, 18888,    16,  6439,  1969,  4021,   280,
         3362,   659,  1338,  2106,  1934, 17662,    93,   441,  1086,  2138,
            1,  2024,  4087, 17981,    16,  2063,   498,  2736,     5, 17662,
        12973,     5,   381,  7721,    16,  4187,  6533,   751,   544,  1030,
         2796,  4862,  5153,  4784,   176, 11844, 15557,   656,  2784,  9396,
         1947,  2371,  2896,  2055,    14,  9871,  6533,   751,   763,  1054,
         6463,  5153,  2825,  3951,     3,  2830,  4531,   729, 16245, 18217,
         2662, 17628, 13572,  2493,    14,  3952,  1917, 16011,  6533,   763,
         2878,  6609,  1900,    16,     3]), 'toke

In [45]:
# check collator: add [MASK]
print(data_collator(dataset.examples))

{'input_ids': tensor([[    2,     4, 11555,  ...,     4,    16,     3],
        [    2,    14,     4,  ..., 10313,    16,     3],
        [    2,  1984,   751,  ...,   291,   176,     3],
        ...,
        [    2,  4350,  2496,  ...,  3065,    16,     3],
        [    2,  1984,  3954,  ...,  9865,    16,     3],
        [    2,  7000,  1931,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0]]), 'next_sentence_label': tensor([0, 0, 0,  ..., 1, 0, 0]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[-100, 2492, 2429,  ..., 1900, -100, -100],
        [-100, -100, 5060,  ..

In [47]:
tokenizer.decode(data_collator(dataset.examples)['input_ids'][0].tolist())

'[CLS]어는 민주당 출신 미국 39 [MASK] 대통령 이다. [MASK] 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 [MASK] 해군에 들어가 전함 · 원자력 [MASK] 잠수함의 승무원으로 [MASK]. 1953년 미국 해군 대위로 예 [MASK]하였고 이후 땅콩 · 면화 [MASK] [UNK] 많은 돈을 벌었다. 그의 별 [MASK] " 땅콩 농부 " 로 알려졌다. 1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 [MASK] [SEP] 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 [MASK] 조지아 지사로 근무했다. [SEP]'

# train model

In [48]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="model_output",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=32,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ipari3 (use `wandb login --relogin` to force relogin)


Step,Training Loss


TrainOutput(global_step=740, training_loss=8.999630531104835, metrics={'train_runtime': 388.7286, 'train_samples_per_second': 1.904, 'total_flos': 1784424819165000.0, 'epoch': 10.0, 'init_mem_cpu_alloc_delta': 1613869056, 'init_mem_gpu_alloc_delta': 406882304, 'init_mem_cpu_peaked_delta': 275832832, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 126128128, 'train_mem_gpu_alloc_delta': 1284682240, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 4736489472})

In [ ]:
trainer.save_model("model_output")

In [ ]:
from transformers import BertForMaskedLM, pipeline

my_model = BertForMaskedLM.from_pretrained("model_output")
nlp_fill = pipeline('fill-mask', top_k=5, model=my_model, tokenizer=tokenizer)
nlp_fill('이순신은 [MASK] 중기의 무신이다.')